In [1]:
import pandas as pd
import numpy as np
import os
import simplejson


In [5]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [2]:
def file_params(df):
    params = {}
    #size
    params['size'] = {}
    params['size']['rows'] = int(df.shape[0])
    params['size']['cols'] = int(df.shape[1])

    #missing values
    params['missing'] = {}
    ##rows
    params['missing']['rows'] = int(df[df.isna().any(axis=1)].shape[0])
    params['missing']['rowsPercent'] = float(round(params['missing']['rows'] / params['size']['rows'], 7) * 100)
    ##cells
    params['missing']['cells'] = int(df.isna().sum().sum())
    params['missing']['cellsPercent'] = float(round(params['missing']['cells'] / (params['size']['rows'] * params['size']['cols']), 7) * 100)
    ##columns
    missingColumn = df.isna().sum()
    params['missing']['cols'] = missingColumn[missingColumn != 0].to_dict()
    ###percent of total values in colums
    nanByColumnPercent = round(df.isna().sum() / df.count().sum() * 100, 4)
    params['missing']['colsPercent'] = nanByColumnPercent[nanByColumnPercent !=0].to_dict()
    ###percent ot total missing values
    nanColumnContributionPercent = round(df.isna().sum() / df.isna().sum().sum() * 100, 2)
    params['missing']['colsPercentContribution'] = nanColumnContributionPercent[nanColumnContributionPercent !=0].to_dict()

    #names
    params['names'] = {}
    params['names']['cols'] = list(df.columns.values)
    params['names']['colsReverse'] = list(df.columns.values)
    params['names']['colsReverse'].reverse()

    #describe
    params['describe'] = df.describe().to_dict()

    return params

In [36]:
path =  '/Users/brandon/Desktop/NFL-Gen-Test_missing.csv'

df = load_file(path)



208

In [120]:
missing = []

#determine how to relabel groupings of booleans
mapDict = {True: 'singleMissing', False: 'multipleMissing'}

#calculate how many rows have single missing values
#row_opporunities = (df[df.isnull().sum(axis=1) != 0].isnull().sum(axis=1) == 1).map(mapDict).value_counts()

#determine metrics for each individula column
for col in df.columns:

    #determine how many rows have single missing values vs multiple
    split = df[df[col].isna()].isna().sum(axis=1) > 1
    d = split.map(mapDict).value_counts() #group

    

    #complete dictionary
    if 'singleMissing' not in d:
        d['singleMissing'] = 0
    if 'multipleMissing' not in d:
        d['multipleMissing'] = 0

    #other metrics to use
    d['total'] = d.sum()
#    d['percentValues'] = round(d['total'] / (df.shape[0] * df.shape[1]) * 100, 2)
#    d['percentContributions'] = round(d['total'] / df.isnull().sum().sum() * 100, 2)
    d['col'] = col

    if d['total'] > 0:
        d = d.to_dict()
        missing.append(d)

print(missing)

     
# drop_calc = {}
# drop_calc['totalRows'] = df.shape[0]
# drop_calc['totalCols'] = df.drop(dropped_cols, axis=1).dropna().shape[0]


[{'multipleMissing': 2, 'singleMissing': 1, 'total': 3, 'col': 'BenchReps'}, {'singleMissing': 1, 'multipleMissing': 0, 'total': 1, 'col': 'Broad Jump'}, {'multipleMissing': 3, 'singleMissing': 0, 'total': 3, 'col': '3Cone'}]


7